In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.express as px
import sklearn
from sklearn.utils import shuffle
import cv2
import os
from tensorflow import keras
from keras import layers

In [2]:
class_names = ['mountain', 'street', 'glacier', 'buildings', 'sea', 'forest']
class_names_label = {class_name: i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (150, 150)

In [3]:
datasets = ['Datasets/SateliteImages/seg_train/seg_train', 'Datasets/SateliteImages/seg_test/seg_test']
output = []

# Iterate through training and test sets
for dataset in datasets:

    images = []
    labels = []

    print("Loading {}".format(dataset))

    # Iterate through each folder corresponding to a category
    for folder in os.listdir(dataset):
        label = class_names_label[folder]

        # Iterate through each image in our folder
        for file in os.listdir(os.path.join(dataset, folder)):
            # Get the path name of the image
            img_path = os.path.join(os.path.join(dataset, folder), file)

            # Open and resize the img
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, IMAGE_SIZE)

            # Append the image and its corresponding label to the output
            images.append(image)
            labels.append(label)

    images = tf.convert_to_tensor(images, dtype='float32')
    labels = tf.convert_to_tensor(labels, dtype='int32')

    output.append((images, labels))
(train_images, train_labels), (test_images, test_labels) = output

Loading Datasets/SateliteImages/seg_train/seg_train


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
# X_train = tf.convert_to_tensor(train_images)
# y_train = tf.convert_to_tensor(train_labels)
# X_dev = tf.convert_to_tensor(test_images)
# y_dev = tf.convert_to_tensor(test_labels)
X_train = train_images
y_train = train_labels
X_dev = test_images
y_dev = test_labels
print("Is there a GPU available: "),
print(tf.config.list_physical_devices("GPU"))

print("Are Tensors on GPU #0:  "),
print([a.device.endswith('GPU:0') for a in [X_train, X_dev, y_train, y_dev]])

In [ ]:
tf.random.set_seed(69)
np.random.seed(69)
hyperOptimizer = tf.keras.optimizers.Adam(learning_rate=0.1, weight_decay=0)
model = tf.keras.Sequential([layers.Conv2D(16, kernel_size=3, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1] , 3)), layers.MaxPool2D(pool_size=2, strides=2), layers.Conv2D(filters=4, kernel_size=3, strides=1), layers.Flatten(), layers.Dense(128, activation=keras.activations.relu), layers.Dense(nb_classes)])
model.compile(optimizer=hyperOptimizer, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=('acc'))
model.summary()

In [ ]:
model.fit(X_train, y_train, batch_size=128, epochs=10,validation_data=(X_dev, y_dev))